In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Solo carga de datos

In [ ]:
data_train = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")
data_test = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")
data_train['label'].unique()

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data_train,train_size=0.8)

data_val.info()
data_train.info()

# Cambios, dejo como 1 mi digito verificador (3) y todo los demas como 0

In [ ]:
data_train['label'].replace(to_replace=[0,1,2,3,4,5,6,7,8,9],value=[0,0,0,1,0,0,0,0,0,0],inplace =True)
data_train['label'].unique()

data_val['label'].replace(to_replace=[0,1,2,3,4,5,6,7,8,9],value=[0,0,0,1,0,0,0,0,0,0],inplace =True)
data_val['label'].unique()

data_test['label'].replace(to_replace=[0,1,2,3,4,5,6,7,8,9],value=[0,0,0,1,0,0,0,0,0,0],inplace =True)
data_test['label'].unique()

# Solo separando datos

In [ ]:
x_train = data_train.iloc[:,1:].values.astype('float32')
y_train = data_train.iloc[:,0].values.astype('int32')

x_val = data_val.iloc[:,1:].values.astype('float32')
y_val = data_val.iloc[:,0].values.astype('int32')

x_test = data_test.iloc[:,1:].values.astype('float32')
y_test = data_test.iloc[:,0].values.astype('int32')

_train son para entrenar

_val son para validar

_test son para probar

In [ ]:
print("Datos para entrenar: ", x_train.shape[0])
print("Datos para validar: ",  x_val.shape[0])
print("Datos para probar: ",   x_test.shape[0])

# Solo visualizacion

In [ ]:
plt.figure(figsize=(20,20))
for i in range(40):
    plt.subplot(8,5,i+1)
    plt.imshow(np.reshape(x_val[i],(28,28)),cmap='gray')
    plt.title("Clase:{}".format(y_val[i]))
    plt.xticks([])
    plt.yticks([])

# Normalizacion (Por ahi lo llaman buenas practicas)

In [ ]:
x_train = x_train/255

x_val = x_val/255

x_test = x_test/255

# Codificacion One-hot

In [ ]:
from keras.utils import np_utils
num_clases = 2
Y_train = np_utils.to_categorical(y_train,num_clases)
Y_val = np_utils.to_categorical(y_val,num_clases)
Y_test = np_utils.to_categorical(y_test,num_clases)
print(y_train.shape, " ",Y_train.shape)
print(y_val.shape, " ",Y_val.shape)
print(y_test.shape, " ",Y_test.shape)

# Basicamente la red neuronal con los parametros que pidio el profe

Epocas: 100

Capas ocultas: 1

Cantidad de neuronas en la capa oculta: 25

Funcion de activacion de capa oculta: sigmoid

Entradas en la capa oculta: 784 (La imagen se compone de 28x28 celdas en el archivo.csv, por lo tanto las columnas en el archivo.csv son 1x1,1x2,1x3...,1x27,1x28,2x1,2x2,2x3,...,2x27,2x28,...,28x1,...,28x27,28x28  y la cantidad de columnas es 784 = 28x28)

Capa de salida: 2 neuronas (Es un 1 o es un 0)

Funcion de activacion en capa de salida: softmax

EarlyStopping: 15 que monitorea loss de validacion (val_loss)

Funcion de perdida: Cross Entropy (Como: **es** 3 o **no es** 3, utilizamos binary_crossentropy, si fueran mas numeros y si utilizamos One-hot encoding se utiliza categorical_crossentropy, el otro ni idea para cuando se utiliza)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras import activations, initializers
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping

Epocas = 100


MLP = Sequential()

#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)

#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))

#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = SGD(lr=0.1)

MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])



In [ ]:
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val)
                  )

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

# Solo visualizacion

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Correctos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict: {}, Real: {}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Incorrectos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict: {}, Real: {}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

# Graficos

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Cambiando funcion objetivo a MSE

In [ ]:
from keras.losses import MeanSquaredError

MLP = Sequential()

#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)

#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))

#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = SGD(lr=0.1)

MLP.compile(loss=MeanSquaredError(), optimizer = optim, metrics=['accuracy'])

In [ ]:
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val)
                  )

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# LR = 0.01

In [ ]:
Epocas = 100
MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = SGD(lr=0.01)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Correctos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict:{}, Real:{}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Incorrectos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict:{}, Real:{}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

# Solo graficos

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# LR = 1

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = SGD(lr=1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# LR = 10

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = SGD(lr=10)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Red neuronal con units = 1

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 1, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=1000)

optim = SGD(lr=0.1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])

Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Red neuronal con units = 10

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 10, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=1000)

optim = SGD(lr=0.1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])

Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Red neuronal con units = 25

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=1000)

optim = SGD(lr=0.1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])

Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Red neuronal con units = 100

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 100, kernel_initializer= pesos_inicial, input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=1000)

optim = SGD(lr=0.1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])

Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Utilizando ADAM

In [ ]:
Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial , input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

optim = Adam(lr=0.1)
MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Correctos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict:{}, Real:{}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

In [ ]:
plt.figure(figsize=(20,20))
for i,nice in enumerate(Incorrectos[:100]):
    plt.subplot(10,10,i+1)
    plt.imshow(x_test[nice].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
              "Predict:{}, Real:{}".format(resultado[nice],
                                            y_test[nice]))
    plt.xticks([])
    plt.yticks([])

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

# Con weight decay (weight regularizer)

In [ ]:
from keras.regularizers import l2

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, kernel_regularizer=l2() , input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=15)

MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")

In [ ]:


Epocas = 100

MLP = Sequential()
#Distribucion para inicializar los pesos
pesos_inicial = initializers.RandomUniform(minval=-0.2,maxval=0.2)
#capa oculta
MLP.add(Dense(units= 25, kernel_initializer= pesos_inicial, kernel_regularizer=l2() , input_dim= 784, activation= activations.sigmoid))
#output
MLP.add(Dense(2, activation = activations.softmax))

earlyStopping = EarlyStopping(monitor='val_loss',patience=1000)

MLP.compile(loss='categorical_crossentropy', optimizer = optim, metrics=['accuracy'])
Historia = MLP.fit( x_train, Y_train, 
                    epochs=Epocas,
                    batch_size=32, #mini batch
                    callbacks=[earlyStopping], #Ojala funcione
                    validation_data = (x_val, Y_val))

In [ ]:
resultado = MLP.predict_classes(x_test)
Correctos = np.nonzero(resultado == y_test)[0]
Incorrectos = np.nonzero(resultado != y_test)[0]

print("Set de entrenamiento: ", y_test.shape[0])
print("Acertados: ",len(Correctos))
print("No acertados: ",len(Incorrectos))

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['loss'], label='training data')
plt.plot(Historia.history['val_loss'], label='validation data')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(Historia.history['accuracy'], label='accuracy')
plt.plot(Historia.history['val_accuracy'], label='validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="upper left")